In [26]:
import numpy as np
import string
from nltk.corpus import stopwords
import csv
import pandas as pd

In [27]:
url = "./datasets/pre_processed_dataset.csv"
names = ["tweet"]
data = pd.read_csv(url,names=names)
data = data.drop(data.index[0])

In [28]:
from nltk.tokenize import word_tokenize
import nltk

sentences = data.tweet

stop_words = nltk.corpus.stopwords.words('english')
data_set_tokenized = []

for i in sentences:
    word_tokens = word_tokenize(i)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    data_set_tokenized.append(filtered_sentence)

data_set_tokenized[0]
# filtered_sentence

['never',
 'know',
 'another',
 'concern',
 'farmed',
 'salmon',
 'gmo',
 'farmed',
 'frankenfish']

## Word2Vec with skipgramc

In [29]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec


In [30]:

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1


In [31]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1


In [32]:
skip_gram_model = Word2Vec(sentences=data_set_tokenized, vector_size=100, window=5, min_count=4, workers=2, epochs=5, sg=1,
        compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 4484.88134765625
Loss after epoch 1: 9166.8349609375
Loss after epoch 2: 14294.6611328125
Loss after epoch 3: 19074.94921875
Loss after epoch 4: 24340.212890625


In [33]:
skip_gram_model.wv.most_similar('crop',topn=10)

[('war', 0.3227420151233673),
 ('chemicals', 0.24044738709926605),
 ('amp', 0.2135903537273407),
 ('news', 0.20882479846477509),
 ('scientists', 0.2075941264629364),
 ('seeds', 0.2042963057756424),
 ('gmo', 0.2026502937078476),
 ('say', 0.20226021111011505),
 ('complete', 0.2018624246120453),
 ('herbicide', 0.19491374492645264)]

In [34]:
skip_gram_model.wv.most_similar('gmo',topn=10)

[('complete', 0.5709086060523987),
 ('amp', 0.5685555338859558),
 ('africa', 0.547189474105835),
 ('liver', 0.5244296789169312),
 ('mineral', 0.5101735591888428),
 ('people', 0.5075904130935669),
 ('fertilizer', 0.49014922976493835),
 ('homeloan', 0.4868205785751343),
 ('new', 0.4725566804409027),
 ('genetic', 0.4709470570087433)]

In [35]:
skip_gram_model.wv.most_similar('corn',topn=10)

[('humans', 0.4742598831653595),
 ('amp', 0.43114569783210754),
 ('pesticides', 0.4251401126384735),
 ('gmo', 0.4157561957836151),
 ('human', 0.3899383246898651),
 ('potential', 0.38027453422546387),
 ('mineral', 0.3507687449455261),
 ('africa', 0.3484934866428375),
 ('stop', 0.33811092376708984),
 ('fertilizer', 0.3309765160083771)]

## Word2Vec with CBOW

In [36]:
cbow_model = Word2Vec(sentences=data_set_tokenized, vector_size=100, window=10, min_count=4, workers=2, epochs=5, sg=0,
        compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 2229.346923828125
Loss after epoch 1: 4533.95263671875
Loss after epoch 2: 6897.97509765625
Loss after epoch 3: 9148.1650390625
Loss after epoch 4: 11455.7548828125


In [37]:
cbow_model.wv.most_similar('crop',topn=10)

[('war', 0.3049982786178589),
 ('chemicals', 0.21580922603607178),
 ('labeling', 0.20013035833835602),
 ('news', 0.1575966775417328),
 ('say', 0.15510551631450653),
 ('also', 0.15459318459033966),
 ('herbicide', 0.1451404094696045),
 ('seeds', 0.1409464329481125),
 ('scientists', 0.1345074325799942),
 ('chestnut', 0.13428443670272827)]

## LSTM